## City of Yellowknife Zones Count

In [1]:
import pandas as pd

In [15]:
zones = pd.read_csv("data/city_of_yellowknife/yk_zones_shp/yk_zones.csv")

zones_count = zones.groupby(by="CODE_SHORT").size().to_frame(name="count")
zones_count = zones_count.reset_index()
zones_count.head(1)

,CODE_SHORT,count
0,AE,2


## Download OSM Data

In [19]:
import pyrosm
from pyrosm import get_data

fp = get_data("northwest_territories", directory = "data/osm")

Downloaded Protobuf data 'northwest-territories-latest.osm.pbf' (196.07 MB) to:
'C:\Users\teres\OneDrive - University of Toronto\Documents\GitHub\yellowknife\analysis\data\osm\northwest-territories-latest.osm.pbf'
